In [10]:
import numpy as np
from time import time
from scipy import sparse
from math import exp

from oracles import BinaryLogistic

In [19]:
class GDClassifier:
    """
    Реализация метода градиентного спуска для произвольного
    оракула, соответствующего спецификации оракулов из модуля oracles.py
    """
    
    def __init__(self, loss_function, step_alpha=1, step_beta=0, 
                 tolerance=1e-5, max_iter=1000, **kwargs):
        """
        loss_function - строка, отвечающая за функцию потерь классификатора. 
        Может принимать значения:
        - 'binary_logistic' - бинарная логистическая регрессия
                |
        step_alpha - float, параметр выбора шага из текста задания
        
        step_beta- float, параметр выбора шага из текста задания
        
        tolerance - точность, по достижении которой, необходимо прекратить оптимизацию.
        Необходимо использовать критерий выхода по модулю разности соседних значений функции:
        если |f(x_{k+1}) - f(x_{k})| < tolerance: то выход 
        
        max_iter - максимальное число итераций     
        
        **kwargs - аргументы, необходимые для инициализации   
        """

        if loss_function != 'binary_logistic':
            raise TypeError

        self.loss = loss_function

        self.step_alpha = step_alpha
        self.step_beta = step_beta

        self.tolerance = tolerance

        self.max_iter = max_iter
        
        self.oracle = BinaryLogistic(l2_coef=kwargs['l2_coef'])
        
        
    def fit(self, X, y, w_0=None, trace=False):
        """
        Обучение метода по выборке X с ответами y
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        y - одномерный numpy array
        
        w_0 - начальное приближение в методе
        
        trace - переменная типа bool
      
        Если trace = True, то метод должен вернуть словарь history, содержащий информацию 
        о поведении метода. Длина словаря history = количество итераций + 1 (начальное приближение)

        history['time']: list of floats, содержит интервалы времени между двумя итерациями метода
        history['func']: list of floats, содержит значения функции на каждой итерации
        (0 для самой первой точки)
        """
        
        def lambda_k(k):
            return self.step_alpha / (k ** self.step_beta)

        
        if w_0 is None:
            w = np.zeros(X.shape[1])
        
        history = {}
        times = []
        funcs = []
        
        cur_func = self.oracle.func(X, y, w)
        funcs.append(cur_func)
        times.append(0)
        
        i = 1
        while i <= self.max_iter:
            
            time_b4 = time()
            
            w_next = w - lambda_k(i) * self.oracle.grad(X, y, w)
            
            time_delta = time() - time_b4
            
            times.append(time_delta)
            
            next_func = self.oracle.func(X, y, w_next)
            funcs.append(next_func)
            
            if abs(next_func - cur_func) < self.tolerance:
                break
                
            w = w_next
            cur_func = next_func
            i += 1
            
        history['time'] = times
        history['func'] = funcs
        
        self.w = w_next
        
        if trace:
            return history
        
    def predict(self, X):
        """
        Получение меток ответов на выборке X
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        return: одномерный numpy array с предсказаниями
        """
        
        ans = np.zeros(X.shape[0]).astype(int)
        
        for enum, item in enumerate(X):
            mul = np.sum(self.w * item)
            proba = 1 / (1 + exp(-1 * mul))
            
            if proba < 0.5:
                ans[enum] = -1
            else:
                ans[enum] = 1
                
        return ans

    def predict_proba(self, X):
        """
        Получение вероятностей принадлежности X к классу k
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        
        return: двумерной numpy array, [i, k] значение соответветствует вероятности
        принадлежности i-го объекта к классу k 
        """
        ans = []
        for enum, item in enumerate(X):
            mul = np.sum(self.w * item)
            proba = 1 / (1 + exp(-1 * mul))
            
            ans.append([proba, 1 - proba])
                
        return np.array(ans)
        
    def get_objective(self, X, y):
        """
        Получение значения целевой функции на выборке X с ответами y
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        y - одномерный numpy array
        
        return: float
        """
        return self.oracle.func(X, y, self.w)
        
    def get_gradient(self, X, y):
        """
        Получение значения градиента функции на выборке X с ответами y
        
        X - scipy.sparse.csr_matrix или двумерный numpy.array
        y - одномерный numpy array
        
        return: numpy array, размерность зависит от задачи
        """
        return self.oracle.grad(X, y, self.w)
    
    def get_weights(self):
        """
        Получение значения весов функционала
        """    
        return self.w


In [4]:
X = (np.random.rand(10)).reshape(2, 5)
y = np.array([1, -1])

In [5]:
classifier = GDClassifier('binary_logistic')

In [6]:
classifier.fit(X, y)

(2, 5)


In [11]:
for item in X:
    print(item)

[0.39024708 0.59416511 0.64308653 0.26849751 0.39084895]
[0.80672398 0.41287117 0.44277221 0.62547913 0.76510014]


In [13]:
Y = X.tolist()

In [18]:
X

array([[0.39024708, 0.59416511, 0.64308653, 0.26849751, 0.39084895],
       [0.80672398, 0.41287117, 0.44277221, 0.62547913, 0.76510014]])